In [1]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
import os
import numpy as np
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import pylab
import skimage.io as io
import json

import re

In [2]:
'''
# show an image file
mmcv.imshow('a.jpg')

# show a loaded image
img = np.random.rand(100, 100, 3)
mmcv.imshow(img)

# show image with bounding boxes
img = np.random.rand(100, 100, 3)
bboxes = np.array([[0, 0, 50, 50], [20, 20, 60, 60]])
mmcv.imshow_bboxes(img, bboxes)
'''

"\n# show an image file\nmmcv.imshow('a.jpg')\n\n# show a loaded image\nimg = np.random.rand(100, 100, 3)\nmmcv.imshow(img)\n\n# show image with bounding boxes\nimg = np.random.rand(100, 100, 3)\nbboxes = np.array([[0, 0, 50, 50], [20, 20, 60, 60]])\nmmcv.imshow_bboxes(img, bboxes)\n"

In [3]:
from pycocotools.coco import COCO

In [4]:
annFile = '/home/user/mmdetection-1.2.0/data/sar_coco_form/SSDD/annotations/train_test2017.json'

In [5]:
coco=COCO(annFile)  

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [6]:
##### COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds()) 
#clw note:输出形如{'id': 1, 'supercategory': 'head', 'name': 'head'}
nms = [cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

COCO categories: 
ship



In [7]:
catIds = coco.getCatIds(catNms=['ship'])
imgIds = coco.getImgIds(catIds=catIds)

In [8]:
len(imgIds)  # 所有图片

1160

In [9]:
img = coco.loadImgs(coco)

In [10]:
def json_parser(anno_root):
    with open(anno_root) as f:
        data = json.load(f)
    
    # info = data['info']
    # licenses = data['licenses']
    
    '''
        print(data.keys()) 
        return: dict_keys(['images', 'annotations', 'categories'])
    '''
    category = [c['name'] for c in data['categories']] # return ship
    category_dict = {c['id']: c['name'] for c in data['categories']}
    # category_dict : {1: 'ship'}
    max_category_id = max(category_dict.keys())
    
    annotations = data['annotations']
    images = data['images']
    
    # id to image mapping
    image_dict = {}
    img_list = list()
    
    for image in images:
        key = image['id']
        image_dict[key] = [image['file_name'], image['width'], image['height']]
        img_list.append(image['file_name'])
        
    category_count = [0 for _ in range(max_category_id)]
    
    total_annotations = {}
    
    for a in annotations:
        image_name = image_dict[a["image_id"]][0].replace('.jpg', '')
        width = image_dict[a["image_id"]][1]
        height = image_dict[a["image_id"]][2]
        idx = a['category_id']
        single_ann = []
        single_ann.append(category_dict[idx])
        single_ann.extend(list(map(int, a['bbox'])))
        single_ann.extend([width, height])
        
        if image_name not in total_annotations:
            total_annotations[image_name] = []
        
        category_count[idx - 1] += 1
        total_annotations[image_name].append(single_ann)
    
    print('\n==============[ COCO json info ]==============')
    print("Total Annotations: {}".format(len(annotations)))
    print("Total Image      : {}".format(len(images)))
    print("Annotated Image  : {}".format(len(total_annotations)))
    print("Total Category   : {}".format(len(category)))
    
    for c, cnt in zip(category, category_count):
        if cnt != 0:
            print("{:^20}| {}".format(c, cnt))
    print()
    
    return category, img_list, total_annotations
            

In [11]:
category, img_list, total_annotations = json_parser(annFile)


==============[ COCO json info ]==============
Total Annotations: 2540
Total Image      : 1160
Annotated Image  : 1160
Total Category   : 1
        ship        | 2540



In [12]:
# annFile_HRSID = '/home/user/mmdetection-1.2.0/data/sar_coco_form/HRSID/annotations/train_test2017.json'
# category, img_list, total_annotations = json_parser(annFile_HRSID)

In [13]:
img_list

['000019.jpg',
 '001067.jpg',
 '000786.jpg',
 '001121.jpg',
 '000364.jpg',
 '000344.jpg',
 '000115.jpg',
 '000698.jpg',
 '001152.jpg',
 '000015.jpg',
 '000219.jpg',
 '000589.jpg',
 '000095.jpg',
 '000533.jpg',
 '000886.jpg',
 '000787.jpg',
 '000450.jpg',
 '000617.jpg',
 '000064.jpg',
 '000027.jpg',
 '000126.jpg',
 '000612.jpg',
 '001138.jpg',
 '000290.jpg',
 '000104.jpg',
 '000398.jpg',
 '000168.jpg',
 '000056.jpg',
 '000912.jpg',
 '000716.jpg',
 '000111.jpg',
 '000025.jpg',
 '000451.jpg',
 '000688.jpg',
 '000678.jpg',
 '000067.jpg',
 '000139.jpg',
 '000394.jpg',
 '000161.jpg',
 '000714.jpg',
 '000265.jpg',
 '001020.jpg',
 '001141.jpg',
 '000839.jpg',
 '001052.jpg',
 '000478.jpg',
 '000737.jpg',
 '000175.jpg',
 '000853.jpg',
 '001151.jpg',
 '001088.jpg',
 '000583.jpg',
 '000321.jpg',
 '000215.jpg',
 '000085.jpg',
 '000503.jpg',
 '000743.jpg',
 '001126.jpg',
 '000988.jpg',
 '000400.jpg',
 '000084.jpg',
 '000498.jpg',
 '000699.jpg',
 '000598.jpg',
 '000149.jpg',
 '000831.jpg',
 '000734.j

In [14]:
total_annotations

{'000019': [['ship', 147, 112, 83, 86, 418.0, 355.0]],
 '001067': [['ship', 122, 173, 17, 12, 488.0, 331.0],
  ['ship', 214, 192, 26, 12, 488.0, 331.0],
  ['ship', 209, 199, 9, 10, 488.0, 331.0],
  ['ship', 239, 139, 22, 13, 488.0, 331.0],
  ['ship', 302, 100, 12, 9, 488.0, 331.0],
  ['ship', 329, 102, 14, 10, 488.0, 331.0]],
 '000786': [['ship', 223, 155, 15, 37, 502.0, 337.0]],
 '001121': [['ship', 42, 86, 26, 15, 510.0, 311.0],
  ['ship', 57, 153, 26, 12, 510.0, 311.0],
  ['ship', 150, 89, 30, 18, 510.0, 311.0],
  ['ship', 140, 195, 27, 16, 510.0, 311.0],
  ['ship', 236, 149, 25, 15, 510.0, 311.0],
  ['ship', 315, 147, 19, 16, 510.0, 311.0],
  ['ship', 286, 91, 24, 15, 510.0, 311.0],
  ['ship', 361, 50, 16, 12, 510.0, 311.0],
  ['ship', 413, 90, 19, 13, 510.0, 311.0],
  ['ship', 404, 164, 20, 13, 510.0, 311.0],
  ['ship', 368, 245, 21, 12, 510.0, 311.0]],
 '000364': [['ship', 45, 275, 15, 35, 500.0, 388.0],
  ['ship', 226, 189, 15, 26, 500.0, 388.0],
  ['ship', 326, 307, 15, 30, 500

In [15]:
def get_singleImg_gt(name, total_annotations):
    if name.replace('.jpg', '') not in total_annotations.keys():
        print('There are no annotations in %s.' % name)
        return []
    else:
        return total_annotations[name.replace('.jpg', '')]


In [16]:
name = '000006.jpg'

In [17]:
bboxes = get_singleImg_gt(name, total_annotations)
bboxes

[['ship', 101, 136, 80, 49, 501.0, 357.0],
 ['ship', 343, 153, 53, 27, 501.0, 357.0],
 ['ship', 266, 242, 13, 60, 501.0, 357.0]]

In [35]:
from PIL import Image, ImageTk

def draw_gt_boxes(img, objs):
    
    img = cv2.imread(img)
    
    for obj in objs:
        box = obj[1:]
        xmin = max(box[0], 0)
        ymin = max(box[1], 0)
        xmax = min(box[0] + box[2], obj[5])
        ymax = min(box[1] + box[3], obj[6])
        
        cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0,0,225), 2)
    
    cv2.imwrite('2.jpg', img)
        
    

In [36]:
img = draw_gt_boxes(name, bboxes)

In [37]:
# cv2.imshow('ss',img)

In [38]:
a =mmcv.imshow(img)

TypeError: "img" must be a numpy array or a str or a pathlib.Path object